In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier
from skimage.io import imread
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, InputLayer
from tensorflow.keras.optimizers import Adam

import os
import glob


load data

In [2]:
# ResNet take RGB images of size 224x224
def load_data_from_folder(folder_path):
    images = []
    labels = []
    for label in ["HC", "PD"]:
        label_path = f"{folder_path}/{label}"
        for image_path in os.listdir(label_path):
            full_path = f"{label_path}/{image_path}"
            try:
                img = imread(full_path)
                # Ensure image is RGB
                if len(img.shape) == 2:
                    img = gray2rgb(img)
                img = resize(img, (224, 224, 3))  # Resize to the expected shape for ResNet50
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error processing image: {full_path}")
                print(f"Error details: {e}")
                continue

    return np.array(images), np.array(labels)

# Load data
drawings_X, drawings_y = load_data_from_folder("drawings")
speech_X, speech_y = load_data_from_folder("speech")


data processing

In [3]:
#resize images
drawings_X_resized = np.array([resize(image, (224, 224)) for image in drawings_X])
speech_X_resized = np.array([resize(image, (224, 224)) for image in speech_X])

# Load the ResNet50 model with weights pre-trained on ImageNet without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# # Extract features
# drawings_features = base_model.predict(preprocess_input(drawings_X_resized))
# speech_features = base_model.predict(preprocess_input(speech_X_resized))


 6/67 [=>............................] - ETA: 29s

: 

train model

In [ ]:
def create_fc_model(input_shape):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=input_shape))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Train the fully connected model for drawings data
drawings_model = create_fc_model((2048,))  # 2048 features from ResNet50 with pooling='avg'
drawings_model.fit(drawings_features, drawings_y, epochs=10, batch_size=32, validation_split=0.2)

# Train the fully connected model for speech data
speech_model = create_fc_model((2048,))
speech_model.fit(speech_features, speech_y, epochs=10, batch_size=32, validation_split=0.2)


evaluate, ensemble

In [ ]:
drawings_model.save('drawings_model.h5')
speech_model.save('speech_model.h5')
